In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from deep_translator import GoogleTranslator
from transformers import pipeline

url = "https://www.cnbce.com/piyasalar"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
post_cards = soup.find_all('div', class_='post-card-title')

titles = []
for post_card in post_cards:
    a_tag = post_card.find('a')
    if a_tag:
        text = a_tag.get_text().strip()
        titles.append(text)

translated_titles = []
for title in titles:
    translated_title = GoogleTranslator(source='auto', target='en').translate(title)
    translated_titles.append(translated_title)

df = pd.DataFrame({'Turkish': titles, 'English': translated_titles})



In [2]:
df

,Turkish,English
0,Haftalık performans: Hangi yatırım aracı ne ka...,Weekly performance: Which investment instrumen...
1,Borsa İstanbul haftayı değer kazanımıyla kapattı,Borsa Istanbul closed the week with value gains
2,İsviçre'nin Türkiye'ye altın ihracatı geriledi,Switzerland's gold exports to Turkey decreased
3,Swap hariç net rezerv 11 milyar dolara çıktı,Net reserves excluding swaps increased to 11 b...
4,Euro Bölgesi PMI'da 8 ay sonra düşüş,Decrease in Euro Zone PMI after 8 months
5,Borsada yatay seyir,Horizontal trend in the stock market
6,Gedik Yatırım model portföyden TTKOM'u çıkarıp...,Gedik Investment removed TTKOM from the model ...
7,50 trilyon euroluk fonu yöneten yatırımcılarla...,He met with investors managing a 50 trillion e...
8,Akbank'tan 500 milyon dolarlık sürdürülebilir ...,500 million dollar sustainable bond issuance f...
9,"ABD'nin ardından İngiltere de ""T+1""i gündemine...","After the USA, England also put ""T+1"" on its a..."


In [3]:
sentiment_analysis = pipeline(
    'sentiment-analysis',
    model='soleimanian/financial-roberta-large-sentiment'
)

config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
results = []
for english_title in df['English']:
    result = sentiment_analysis(english_title)
    results.append(result[0])

df['Label'] = [res['label'].title() for res in results]
df['Score'] = [res['score'] for res in results]

In [5]:
def color_label(val):
    color = ''
    if val == 'Positive':
        color = 'background-color: skyblue; color: black'
    elif val == 'Negative':
        color = 'background-color: lightcoral; color: black'
    elif val == 'Neutral':
        color = 'background-color: lightgrey; color: black'
    return color

styled_df = df.style.applymap(color_label, subset=['Label'])

/var/folders/fh/r8l5j_5x0n7bm87k3t9jsmvw0000gn/T/ipykernel_21299/769962909.py:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(color_label, subset=['Label'])


In [6]:
styled_df

,Turkish,English,Label,Score
0,Haftalık performans: Hangi yatırım aracı ne kazandırdı?,Weekly performance: Which investment instrument gained what?,Neutral,0.998599
1,Borsa İstanbul haftayı değer kazanımıyla kapattı,Borsa Istanbul closed the week with value gains,Positive,0.998199
2,İsviçre'nin Türkiye'ye altın ihracatı geriledi,Switzerland's gold exports to Turkey decreased,Negative,0.998590
3,Swap hariç net rezerv 11 milyar dolara çıktı,Net reserves excluding swaps increased to 11 billion dollars,Positive,0.998201
4,Euro Bölgesi PMI'da 8 ay sonra düşüş,Decrease in Euro Zone PMI after 8 months,Negative,0.998688
5,Borsada yatay seyir,Horizontal trend in the stock market,Neutral,0.998237
6,Gedik Yatırım model portföyden TTKOM'u çıkarıp DOHOL'u aldı,Gedik Investment removed TTKOM from the model portfolio and bought DOHOL,Positive,0.997446
7,50 trilyon euroluk fonu yöneten yatırımcılarla görüştü,He met with investors managing a 50 trillion euro fund,Neutral,0.994739
8,Akbank'tan 500 milyon dolarlık sürdürülebilir tahvil ihracı,500 million dollar sustainable bond issuance from Akbank,Positive,0.995949
9,"ABD'nin ardından İngiltere de ""T+1""i gündemine aldı","After the USA, England also put ""T+1"" on its agenda",Positive,0.987450
